In [ ]:
import time


# BankAccount Class: Manages operations for individual user accounts
class BankAccount:
    def __init__(self, account_number, account_holder, pin, account_type):
        self.account_number = account_number
        self.account_holder = account_holder
        self.balance = 0
        self.transactions = []
        self.pin = pin
        self.account_type = account_type  # Account type ('savings' or 'current')

        # Define interest rate for savings account (if applicable)
        self.interest_rate = 0.02 if self.account_type == 'savings' else 0

    def verify_pin(self, pin):
        """Check if entered PIN matches account's PIN."""
        return self.pin == pin

    def log_transaction(self, description):
        """Log each transaction with a timestamp."""
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        self.transactions.append(f"{timestamp} - {description}")

    def deposit(self, amount, pin):
        """Deposit amount into the account after PIN validation."""
        if self.verify_pin(pin):
            if amount > 0:
                self.balance += amount
                self.log_transaction(f"Deposited: ${amount}")
                print(f"\nDeposit of ${amount} was successful. New Balance: ${self.balance}")
            else:
                print("\nDeposit amount must be greater than 0.")
        else:
            print("\nIncorrect PIN! Deposit failed.")

    def withdraw(self, amount, pin):
        """Withdraw amount from the account after PIN validation."""
        if self.verify_pin(pin):
            if amount > 0 and self.balance >= amount:
                self.balance -= amount
                self.log_transaction(f"Withdrew: ${amount}")
                print(f"\nWithdrawal of ${amount} was successful. New Balance: ${self.balance}")
            else:
                print("\nInsufficient funds or invalid withdrawal amount.")
        else:
            print("\nIncorrect PIN! Withdrawal failed.")

    def check_balance(self, pin):
        """Display the account balance after PIN validation."""
        if self.verify_pin(pin):
            print(f"\nYour current balance is: ${self.balance}")
        else:
            print("\nIncorrect PIN! Unable to display balance.")

    def print_statement(self, pin):
        """Print transaction history after PIN validation."""
        if self.verify_pin(pin):
            print(f"\n--- Transaction History for {self.account_holder} ---")
            if self.transactions:
                for transaction in self.transactions:
                    print(transaction)
            else:
                print("No transactions to display.")
        else:
            print("\nIncorrect PIN! Unable to access transaction history.")

    def calculate_interest(self):
        """Calculate and add interest for savings accounts."""
        if self.account_type == 'savings':
            interest = self.balance * self.interest_rate
            self.balance += interest
            self.log_transaction(f"Interest Added: ${interest}")
            print(f"\nInterest of ${interest} has been added to your savings account.")


# Bank Class: Manages multiple accounts and admin operations
class Bank:
    def __init__(self):
        self.accounts = {}
        self.admin_pin = "1234"  # Predefined admin PIN

    def authenticate_admin(self):
        """Authenticate admin with PIN."""
        entered_pin = input("\nEnter Admin PIN to proceed: ")
        if entered_pin == self.admin_pin:
            return True
        else:
            print("\nInvalid Admin PIN!")
            return False

    def create_account(self, account_holder, account_type):
        """Create a new account for a user."""
        account_number = len(self.accounts) + 1
        pin = self.set_pin(account_holder)
        new_account = BankAccount(account_number, account_holder, pin, account_type)
        self.accounts[account_number] = new_account
        print(f"\nAccount created successfully for {account_holder}!\nAccount Number: {account_number}")
        return new_account

    def set_pin(self, account_holder):
        """Set a secure 4-digit PIN for the account."""
        pin = input(f"\nSet a 4-digit PIN for {account_holder}: ")
        while len(pin) != 4 or not pin.isdigit():
            print("\nInvalid PIN! Please enter a 4-digit PIN.")
            pin = input(f"\nSet a 4-digit PIN for {account_holder}: ")
        return pin

    def get_account(self, account_number):
        """Fetch an account by its number."""
        return self.accounts.get(account_number)

    def total_balance(self):
        """Calculate the total balance across all accounts."""
        total = sum(account.balance for account in self.accounts.values())
        return total

    def total_accounts(self):
        """Get the total number of accounts in the bank."""
        return len(self.accounts)

    def transfer_funds(self, sender_acc_num, receiver_acc_num, amount, pin):
        """Transfer funds between two accounts after validating PIN."""
        sender = self.get_account(sender_acc_num)
        receiver = self.get_account(receiver_acc_num)

        if sender and receiver:
            sender.withdraw(amount, pin)
            receiver.deposit(amount, pin)
            print(f"\nFunds Transfer Successful: ${amount} from Account {sender_acc_num} to Account {receiver_acc_num}")
        else:
            print("\nInvalid account numbers. Transfer failed.")


# Main user interface to interact with the banking system
def banking_interface():
    bank = Bank()  # Create bank instance
    print("\nWelcome to the Python Bank! Your trusted financial partner.\n")
    
    while True:
        try:
            # Menu format as requested
            print("\n--- Main Menu ---")
            print("Open New Account [O]")
            print("Deposit Money [D]")
            print("Withdraw Money [W]")
            print("Check Account Balance [B]")
            print("Transfer Funds [T]")
            print("View Transaction History [H]")
            print("Admin: View Total Deposits [A]")
            print("Admin: View Total Accounts [C]")
            print("Exit [E]")
            
            choice = input("\nEnter your choice: ").strip().upper()

            if choice == "O":  # Open New Account
                holder = input("\nEnter the account holder's name: ")
                account_type = input("Enter account type ('savings' or 'current'): ").lower()
                while account_type not in ['savings', 'current']:
                    print("\nInvalid account type. Please enter either 'savings' or 'current'.")
                    account_type = input("Enter account type ('savings' or 'current'): ").lower()
                bank.create_account(holder, account_type)

            elif choice == "D":  # Deposit Money
                try:
                    account_num = int(input("\nEnter your account number: "))
                    pin = input("Enter your PIN: ")
                    amount = float(input("Enter the deposit amount: $"))
                    account = bank.get_account(account_num)
                    if account:
                        account.deposit(amount, pin)
                    else:
                        print("\nAccount not found.")
                except ValueError as e:
                    print(f"\nInvalid input: {e}")

            elif choice == "W":  # Withdraw Money
                try:
                    account_num = int(input("\nEnter your account number: "))
                    pin = input("Enter your PIN: ")
                    amount = float(input("Enter the withdrawal amount: $"))
                    account = bank.get_account(account_num)
                    if account:
                        account.withdraw(amount, pin)
                    else:
                        print("\nAccount not found.")
                except ValueError as e:
                    print(f"\nInvalid input: {e}")

            elif choice == "B":  # Check Account Balance
                try:
                    account_num = int(input("\nEnter your account number: "))
                    pin = input("Enter your PIN: ")
                    account = bank.get_account(account_num)
                    if account:
                        account.check_balance(pin)
                    else:
                        print("\nAccount not found.")
                except ValueError as e:
                    print(f"\nInvalid input: {e}")

            elif choice == "T":  # Transfer Funds
                try:
                    sender_acc_num = int(input("\nEnter sender account number: "))
                    sender_pin = input("Enter sender PIN: ")
                    receiver_acc_num = int(input("Enter receiver account number: "))
                    amount = float(input("Enter the transfer amount: $"))
                    bank.transfer_funds(sender_acc_num, receiver_acc_num, amount, sender_pin)
                except ValueError as e:
                    print(f"\nInvalid input: {e}")

            elif choice == "H":  # View Transaction History
                try:
                    account_num = int(input("\nEnter your account number: "))
                    pin = input("Enter your PIN: ")
                    account = bank.get_account(account_num)
                    if account:
                        account.print_statement(pin)
                    else:
                        print("\nAccount not found.")
                except ValueError as e:
                    print(f"\nInvalid input: {e}")

            elif choice == "A":  # Admin: View Total Deposits
                if bank.authenticate_admin():
                    print(f"\nTotal Deposits in the Bank: ${bank.total_balance()}")

            elif choice == "C":  # Admin: View Total Accounts
                if bank.authenticate_admin():
                    print(f"\nTotal Number of Accounts in the Bank: {bank.total_accounts()}")

            elif choice == "E":  # Exit
                print("\nThank you for using Python Bank. Goodbye!")
                break

            else:
                print("\nInvalid option. Please select a valid choice.")

        except Exception as e:
            print(f"\nAn unexpected error occurred: {e}")


# Run the banking interface
if __name__ == "__main__":
    banking_interface()



Welcome to the Python Bank! Your trusted financial partner.


--- Main Menu ---
Open New Account [O]
Deposit Money [D]
Withdraw Money [W]
Check Account Balance [B]
Transfer Funds [T]
View Transaction History [H]
Admin: View Total Deposits [A]
Admin: View Total Accounts [C]
Exit [E]
